In [28]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json5 as json
import orjson
# import json
import wandb
import os

In [2]:
project_name = "kyledevinobrien1/In-Context Domain Transfer Improves Out-of-Domain Robustness"
seeds = [3, 17, 46, 58]
# seeds = [3, 17]
seen_paths = set()
all_table_paths = {}
for seed in seeds:
    print(f"Seed: {seed}")
    seed_table_path = json.load(open(f"wandb_paths/wandb_path_seed={seed}.json"))
    
    # Ensure there are no duplicate W&B artifacts
    for path in seed_table_path.values():
        if path is None or path == "":
            continue 
        elif path in seen_paths:
            raise ValueError(f"Seed={seed} - Already seen: {path}")
        else:
            seen_paths.add(path)

    all_table_paths[seed] = seed_table_path
    # print(json.dumps(seed_table_path, indent=4))

all_table_paths

Seed: 3
Seed: 17
Seed: 46
Seed: 58


{3: {'BOSS_Sentiment_ID_BERT_Insert': '',
  'BOSS_Sentiment_ID_BERT_Substitute': '',
  'BOSS_Sentiment_ID_BERT_Translate': '',
  'BOSS_Sentiment_ID_BERT_Paraphrase': '',
  'BOSS_Sentiment_ID_BERT_ICR': '',
  'BOSS_Sentiment_ID_T5_Insert': '',
  'BOSS_Sentiment_ID_T5_Substitute': '',
  'BOSS_Sentiment_ID_T5_Translate': '',
  'BOSS_Sentiment_ID_T5_Paraphrase': '',
  'BOSS_Sentiment_ID_T5_ICR': '',
  'BOSS_Sentiment_ID_Falcon_Insert': '',
  'BOSS_Sentiment_ID_Falcon_Substitute': '',
  'BOSS_Sentiment_ID_Falcon_Translate': '',
  'BOSS_Sentiment_ID_Falcon_Paraphrase': '',
  'BOSS_Sentiment_ID_Falcon_ICR': '',
  'BOSS_Sentiment_SST5_BERT_Insert': 'run-vg726nl1-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0',
  'BOSS_Sentiment_SST5_BERT_Substitute': 'run-vg726nl1-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0',
  'BOSS_Sentiment_SST5_BERT_Translate': 'run-ij6dy9mn-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseunca

In [3]:
run = wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kyledevinobrien1. Use `wandb login --relogin` to force relogin


In [4]:
def get_local_logs_path(directory):
    try:
        for contents in os.listdir(directory):
            if contents.endswith("style_logs.table.json"):
                return os.path.join(directory, contents)

            if os.path.isdir(os.path.join(directory, contents)):
                return get_local_logs_path(os.path.join(directory, contents))
    except:
        # print("No style logs found in {}".format(directory))
        return None

print(get_local_logs_path("data/BOSS_Toxicity_ID_Falcon_Paraphrase"))

None


In [5]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for seed in all_table_paths:
    local_paths[seed] = {}
    for table_name, table_path in tqdm(list(all_table_paths[seed].items())[:]):
        try:
            if table_path == "":
                missing_tables.append(table_name)
                continue

            if get_local_logs_path(f"data/{seed}/{table_name}") is None:
                download_path = f"{project_name}/{table_path}"
                print(f"Downloading {table_name} --- {download_path}")
                artifact = run.use_artifact(download_path, type="run_table")
                artifact_dir = artifact.download(root=f"data/{seed}/{table_name}")

            local_paths[seed][table_name] = get_local_logs_path(f"data/{seed}/{table_name}")
        except Exception as e:
            print(e)
            print(table_name)
            missing_tables.append(table_name)

local_paths

 69%|██████▉   | 224/325 [00:14<00:02, 35.80it/s]

wandb:   1 of 1 files downloaded.  


 70%|██████▉   | 226/325 [00:33<00:55,  1.79it/s]

wandb:   1 of 1 files downloaded.  
 70%|██████▉   | 227/325 [00:37<01:10,  1.38it/s]

wandb:   1 of 1 files downloaded.  


...

wandb: Downloading large artifact run-ryd9xvt6-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0, 193.69MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.7
 99%|█████████▉| 322/325 [06:00<00:06,  2.15s/it]

wandb: Downloading large artifact run-ryd9xvt6-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0, 193.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.7
 99%|█████████▉| 323/325 [06:03<00:04,  2.22s/it]

wandb: Downloading large artifact run-ryd9xvt6-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0, 198.78MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.7
100%|█████████▉| 324/325 [06:05<00:02,  2.25s/it]

wandb: Downloading large artifact run-ryd9xvt6-test_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0, 213.18MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.8
100%|██████████| 325/325 [06:07<00:00,  1.13s/it]


{3: {'BOSS_Sentiment_SST5_BERT_Insert': 'data/3/BOSS_Sentiment_SST5_BERT_Insert/sst5_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Substitute': 'data/3/BOSS_Sentiment_SST5_BERT_Substitute/sst5_aug_substitute_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Translate': 'data/3/BOSS_Sentiment_SST5_BERT_Translate/sst5_aug_back-translate_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Paraphrase': 'data/3/BOSS_Sentiment_SST5_BERT_Paraphrase/sst5_stabilityai/StableBeluga-7b_random_0_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_ICR': 'data/3/BOSS_Sentiment_SST5_BERT_ICR/sst5_stabilityai/StableBeluga-7b_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert': 'data/3/BOSS_Sentiment_Ablate_Data_SST

In [6]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "generations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [7]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

['too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.', 'one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.', 'too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.', 'too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.']
4


In [8]:
local_paths

{3: {'BOSS_Sentiment_SST5_BERT_Insert': 'data/3/BOSS_Sentiment_SST5_BERT_Insert/sst5_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Substitute': 'data/3/BOSS_Sentiment_SST5_BERT_Substitute/sst5_aug_substitute_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Translate': 'data/3/BOSS_Sentiment_SST5_BERT_Translate/sst5_aug_back-translate_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_Paraphrase': 'data/3/BOSS_Sentiment_SST5_BERT_Paraphrase/sst5_stabilityai/StableBeluga-7b_random_0_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_SST5_BERT_ICR': 'data/3/BOSS_Sentiment_SST5_BERT_ICR/sst5_stabilityai/StableBeluga-7b_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert': 'data/3/BOSS_Sentiment_Ablate_Data_SST

In [27]:
combined_datset = DatasetDict()
for seed in local_paths:
    print(f"Seed: {seed}")
    # for split in tqdm([list(local_paths[seed].keys())[0]]):
    for split in tqdm(local_paths[seed]):
        path = local_paths[seed][split]
        # json_logs = json.load(open(path))
        json_logs = orjson.loads(open(path, encoding="utf-8").read())
        frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
        frame = frame.rename(columns=column_renaming)
        frame = frame.drop(columns=columns_to_drop)
        frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

        if "BERT" not in split:
            for col in columns_order:
                if col not in frame.columns:
                    if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                        frame[col] = float("nan")
                    elif col == "prediction_entropy_decreased":
                        frame[col] = False
                    elif col == "tta_all_class_probs":
                        frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                    elif "probs" in col:
                        frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                    elif col == "tta_all_class_entropy":
                        frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)

        if "generations" not in frame.columns:
            frame["generations"] = [[""]] * len(frame)

        frame = frame[columns_order]
        formatted_split_name = f"seed={seed}_{split}"
        combined_datset[formatted_split_name] = Dataset.from_pandas(frame)

combined_datset

Seed: 3


100%|██████████| 280/280 [02:59<00:00,  1.56it/s]


Seed: 17


100%|██████████| 280/280 [02:30<00:00,  1.86it/s]


Seed: 46


100%|██████████| 270/270 [02:29<00:00,  1.81it/s]


Seed: 58


100%|██████████| 265/265 [02:28<00:00,  1.79it/s]


DatasetDict({
    seed=3_BOSS_Sentiment_SST5_BERT_Insert: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 1068
    })
    seed=3_BOSS_Sentiment_SST5_BERT_Substitute: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 1068
    })
    seed=3_BOSS_Sentiment_SST5_BERT_Translate: Dataset({
        features: ['outcome', 'original_tex

In [29]:
combined_datset.save_to_disk("data/combined_dataset_multiseed")

Saving the dataset (1/1 shards): 100%|██████████| 7602/7602 [00:00<00:00, 182584.51 examples/s]


In [30]:
list(combined_datset.keys())

['seed=3_BOSS_Sentiment_SST5_BERT_Insert',
 'seed=3_BOSS_Sentiment_SST5_BERT_Substitute',
 'seed=3_BOSS_Sentiment_SST5_BERT_Translate',
 'seed=3_BOSS_Sentiment_SST5_BERT_Paraphrase',
 'seed=3_BOSS_Sentiment_SST5_BERT_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000

In [34]:
assert len(combined_datset["seed=3_BOSS_Toxicity_Toxigen_BERT_Insert"].features) == len(combined_datset["seed=3_BOSS_Toxicity_Toxigen_T5_Insert"].features)
for feature, value_data in combined_datset["seed=3_BOSS_Toxicity_Toxigen_BERT_Insert"].features.items():
    bert_dtype = value_data.dtype
    t5_dtype = combined_datset["seed=3_BOSS_Toxicity_Toxigen_T5_Insert"].features[feature].dtype
    error_message = f"Feature {feature} has different dtypes for BERT and T5: {bert_dtype} and {t5_dtype}"
    assert bert_dtype == t5_dtype, error_message

In [ ]:
combined_datset["BOSS_Sentiment_ID_BERT_ICR"].features

In [ ]:
combined_datset["BOSS_Sentiment_ID_T5_Insert"].features

In [ ]:
# combined_datset.push_to_hub("Kyle1668/LLM-TTA-Augmentation-Logs")

In [ ]:
# combined_datset.push_to_hub("Kyle1668/LLM-TTA-Augmentation-Logs")


ValueError: All datasets in `DatasetDict` should have the same features but features for 'BOSS_Sentiment_ID_BERT_ICR' and 'BOSS_Sentiment_ID_T5_Insert' don't match: 

{'outcome': Value(dtype='string', id=None), 
'original_text': Value(dtype='string', id=None), 
'augmentations': Sequence(feature=Value(dtype='string', id=None),length=-1, id=None), 'generations': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None), 'original_predicted_class': Value(dtype='int64', id=None), 'tta_predicted_class': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'tta_inference_latency': Value(dtype='float64', id=None), 'original_prediction_entropy': Value(dtype='float64', id=None), 'tta_prediction_entropy': Value(dtype='float64', id=None), 'prediction_entropy_decreased': Value(dtype='bool', id=None), 'prediction_entropy_decrease': Value(dtype='float64', id=None), 'tta_mean_class_probs': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'tta_all_class_probs': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None), 'tta_all_class_entropy': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)} != 

{'outcome': Value(dtype='string', id=None), 'original_text': Value(dtype='string', id=None), 'augmentations': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'generations': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'original_predicted_class': Value(dtype='int64', id=None), 'tta_predicted_class': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'tta_inference_latency': Value(dtype='float64', id=None), 'original_prediction_entropy': Value(dtype='float64', id=None), 'tta_prediction_entropy': Value(dtype='float64', id=None), 'prediction_entropy_decreased': Value(dtype='bool', id=None), 'prediction_entropy_decrease': Value(dtype='float64', id=None), 'tta_mean_class_probs': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'tta_all_class_probs': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None), 'tta_all_class_entropy': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}"